In [1]:

import os, sys, glob, subprocess, pathlib, random
import numpy as np
import pandas as pd
import xarray as xr
from mo_evaluation import mo_evaluate
import mo_evaluation_nonstandard
from MOASMO_parameters import get_parameter_from_Namelist_or_lndin
from xml_addcpubind import insert_cpu_bind

def read_parameter_csv(file_parameter_list):
    df_calibparam = pd.read_csv(file_parameter_list)

    for c in ['Upper', 'Lower', 'Factor', 'Value']:
        if c in df_calibparam.columns:
            if isinstance(df_calibparam.iloc[0][c], str):
                arr = []
                for i in range(len(df_calibparam)):
                    vi = df_calibparam.iloc[i][c]
                    if ',' in vi:
                        arr.append(np.array(vi.split(',')).astype(np.float64))
                    elif '[' in vi:
                        arr.append(np.array(vi.strip('[]').replace('\n', '').split(), dtype=np.float64))
                    else:
                        try:
                            arr.append(np.array([np.float64(vi)]))
                        except:
                            arr.append(np.array([-99999]))
                df_calibparam[c] = arr

    return df_calibparam

def xmlquery_output(pathCTSM, keyword):
    os.chdir(pathCTSM)
    out = subprocess.run(f'./xmlquery {keyword}', shell=True, capture_output=True)
    out = out.stdout.decode().strip().split(' ')[-1]
    return out


def clone_CTSM_model_case(script_clone, source_modelcase, target_modelcase, target_cimeoutput=''):
    if len(target_cimeoutput) > 0:
        settings = f'--cime-output-root {target_cimeoutput}'
    else:
        settings = ''
    _ = subprocess.run(f"{script_clone} --case {target_modelcase} --clone {source_modelcase} {settings} --keepexe", shell=True)
    os.chdir(target_modelcase)
    _ = subprocess.run('./xmlchange BUILD_COMPLETE=TRUE', shell=True)


def update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam):
    # param_names and param_values: list
    file_param_base = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('paramfile'):
    #             file_param_base = line.split('=')[-1].strip().replace('\'', '')
    if len(file_param_base) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('paramfile'):
                    file_param_base = line.split('=')[-1].strip().replace('\'', '')

    ds_param = xr.load_dataset(file_param_base)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_param.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_param_base}!!!')
            sys.exit()
        else:
            vold = ds_param[pn].values
            print(f'  -- Updating parameter {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_param[pn].values.size>1:
                print(f'Warning! New parameter size=1, but raw parameter size={ds_param[pn].values.size}. Force the mean equal to the new parameter value.')
                m = np.nanmean(ds_param[pn].values)
                ds_param[pn].values = ds_param[pn].values - m + np.squeeze(vnew)
            else:
                ds_param[pn].values = np.squeeze(vnew)
    # write to new parameter file
    ds_param.to_netcdf(outfile_newparam, format='NETCDF3_CLASSIC')

def update_CTSM_parameter_NamelistFile(param_names, param_values, file_namelist):
    if len(param_names) > 0:
        with open(file_namelist, 'a') as f:
            print('# Writing new parameter values')
            for i in range(len(param_names)):
                    _ = f.write(f'{param_names[i]} = {np.squeeze(param_values[i])}\n')
                    print(f'  -- Writing parameter {param_names[i]} to namelist file. New value is {param_values[i]}')


def update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata):

    file_surfdata = ''
    # file_user_nl_clm = f'{path_CTSM_clone}/user_nl_clm'
    # with open(file_user_nl_clm) as f:
    #     for line in f:
    #         line = line.strip()
    #         if line.startswith('fsurdat'):
    #             file_surfdata = line.split('=')[-1].strip().replace('\'', '')
    if len(file_surfdata) == 0:
        infile_lndin = f'{path_CTSM_case}/Buildconf/clmconf/lnd_in'
        with open(infile_lndin, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('fsurdat'):
                    file_surfdata = line.split('=')[-1].strip().replace('\'', '')

    ds_surf = xr.load_dataset(file_surfdata)
    for i in range(len(param_names)):
        pn = param_names[i]
        vnew = param_values[i]
        if not pn in ds_surf.data_vars:
            print(f'Error!!! Variable {pn} is not find in parameter file {file_surfdata}!!!')
            sys.exit()
        else:
            vold = ds_surf[pn].values
            print(f'  -- Updating surfdata {pn}: old mean value is {np.nanmean(vold)}. New mean value {np.nanmean(vnew)}')
            if vnew.size==1 and ds_surf[pn].values.size>1:
                print(f'Warning! New surfdata size=1, but raw surfdata size={ds_surf[pn].values.size}. Force the mean equal to the new surfdata value.')
                m = np.nanmean(ds_surf[pn].values)
                ds_surf[pn].values[:] = ds_surf[pn].values - m + np.squeeze(vnew)
            else:
                ds_surf[pn].values[:] = np.squeeze(vnew)

    ds_surf.to_netcdf(outfile_newsurfdata, format='NETCDF3_CLASSIC')


def update_ctsm_parameters(path_CTSM_case, file_parameter_set):

    file_user_nl_clm = f'{path_CTSM_case}/user_nl_clm'
    df_calibparam = pd.read_pickle(file_parameter_set)
    #df_calibparam = read_parameter_csv(file_parameter_set)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Namelist']
    if len(dfi) > 0:
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_NamelistFile(param_names, param_values, file_user_nl_clm)

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Param']
    if len(dfi) > 0:
        outfile_newparam = f'{path_CTSM_case}/updated_parameter.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_ParamFile(param_names, param_values, path_CTSM_case, outfile_newparam)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"paramfile='{outfile_newparam}'\n")

    dfi = df_calibparam.loc[df_calibparam['Source'] == 'Surfdata']
    if len(dfi) > 0:
        outfile_newsurfdata = f'{path_CTSM_case}/updated_surfdata.nc'
        param_names = dfi['Parameter'].values
        param_values = dfi['Value'].values
        update_CTSM_parameter_SurfdataFile(param_names, param_values, path_CTSM_case, outfile_newsurfdata)
        # add to the name list file
        with open(file_user_nl_clm, 'a') as f:
            f.write(f"fsurdat='{outfile_newsurfdata}'\n")

    print('Successfully update parameters!')

In [10]:

if __name__ == '__main__':

    # input arguments
    script_clone = '/glade/u/home/guoqiang/CTSM_repos/CTSM_hillslope/cime/scripts/create_clone'
    path_CTSM_base = '/glade/work/guoqiang/CTSM_CAMELS/Calib_HH_MOASMO/level1_0'
    file_parameter_set = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/param_sets/paramset_iter0_trial324.pkl'
    path_archive = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/ctsm_outputs'
    caseflag = 'iter0_trial3'
    date_start = '2009-10-01' 
    date_end = '2014-10-01'
    ref_streamflow = '/glade/work/guoqiang/CTSM_CAMELS/CAMLES_Qobs/01013500_streamflow_qc.txt'
    add_flow_file = 'NA'

    cpubind_path = '/glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/run_model/iter0/batch0'
    cpuuse = 'Automatic'

    delete_clone = True
    overwrite_previous = False # delete previous simulation results in the archive folder
    

In [11]:
    ########################################################################################################################
    # check whether previous simulation exists
    
    simufiles = glob.glob(f'{path_archive}/{caseflag}/lnd/hist/*.nc')
    if len(simufiles) > 0 and overwrite_previous == False:
        print(f'There are .nc files in {path_archive}/{caseflag}/lnd/hist. No need to run model again')
        runmodel = False
    else:
        runmodel = True
    
    print(runmodel)
    path_CTSM_base = str(pathlib.Path(path_CTSM_base))

There are .nc files in /glade/campaign/cgd/tss/people/guoqiang/CTSM_CAMELS_proj/Calib_HH_MOASMO/level1_0_MOASMOcalib/ctsm_outputs/iter0_trial3/lnd/hist. No need to run model again
False
